# Initialisierung

In [1]:
# 1. Import packages for data analysis
import pandas as pd
import numpy as np          
import matplotlib.pyplot as plt
import os
import zipfile
import openpyxl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from  IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

# 2. Import packages for time analysis
import time
import datetime as dt
from datetime import date
from datetime import timedelta
from datetime import datetime

# 3 Open Teradata Connection. Import packages for time analysis
from pnp_connectors.connection_config_holders import TeradataConfig
from pnp_connectors.quick_create import QuickCreate

# 3.1 create config
td_config = TeradataConfig(
 host="TDP-N0201.deutschepost.dpwn.com",
 secret_user_name=os.getenv("TERADATA_USER"),
 secret_password=os.getenv("TERADATA_PASSWORD")
)

# 3.2 create connection
td_connection = QuickCreate.create_teradata_connection(td_config=td_config)

# 4. Funktion zum Löschen nicht mehr benötigter Tabellen
def drop_table(tab_name):
    try:
        td_connection.execute_sql(f'''DROP TABLE {tab_name}''')
        print(f'''{tab_name} dropped.''')
    except: 
        print(f'''{tab_name} nicht vorhanden''')
         
def create_table(Tabelle, SQL, Index):
    # Falls notwendig löschen der neuen Tabelle, dann Tabelle ohne Daten erstellen und Daten einfügen:
    drop_table(Tabelle)
    td_connection.execute_sql(f'''CREATE MULTISET TABLE {Tabelle} AS ( {SQL}) WITH NO DATA PRIMARY INDEX({Index})''')
    print(Tabelle + ' initialised with PI ' + Index)
    td_connection.execute_sql(f'''INSERT INTO  {Tabelle} {SQL}''')
    test = td_connection.select(f'''select count(1) as anzahl_ds from {Tabelle}''')
    anzahl = str(test['anzahl_ds'].values[0])
    print(f'''{anzahl} DS injected into table {Tabelle}.''')
    print(' ')
    
# 5. test connection
result = td_connection.select("select * FROM DB_NEXTT_DWH.vw_event_zust where sendungs_code = '340434197332847059'")
result.head()

,SENDUNGS_CODE,MODALITAET,EREIGNIS_DATUM,EREIGNIS_ZEITSTEMPEL,LEIT_PLZ,LEIT_STR,LEIT_HNR,LEIT_PRODUKT_CODE,EPOS_KASSEN_NR,FILIAL_DIREKTIONS_NR,FILIAL_NIEDERLASSUNGS_NR,FILIAL_NR,FILIAL_STRASSEN_NAME,FILIAL_PLZ,FILIAL_ORTS_NAME,EPOS_VORGANGSART,EREIGNISTYP,ZIEL_PZ,ZIEL_ZUSTELLBASIS,ZIEL_ZUSTELLBEZIRK,ZIEL_ZUSTELL_ABSCHNITT,SENDUNGSART,EREIGNIS_JAHR,LISTENTYP,LISTEN_NR,LISTEN_POSITION,BENACHRICHTIGUNGS_IDC,NACHNAHME_BETRAG,NACHNAHME_ZAHLUNGSART,ENTGELT_BETRAG,ENTGELT_ZAHLUNGSART,ZOLLEINFUHR_BETRAG,ZAHLSCHEINENTGELT_KENNUNG,WAEHRUNG,STORNOKENNZEICHNUNG,SAMMELBEHAELTER_NR,FOLGETAG,EMPFAENGER,BETRIEBS_NR,ZUSTELLER_NAME,EMPFANGSKENNZEICHNUNG,EMPFAENGER_NAME,AUSWEIS_NR,AUSWEISBEHOERDEN_NAME,AUSWEISKENNZEICHNUNG,AUSWEISGUELTIGKEIT,ERSATZEMPFAENGER_NAME,HSC_ERROR_CODE,PROCESS_ID,DATA_SOURCE_ID,LINE_NO,LOAD_TIMESTAMP,DATA_SOURCE_TYPE_ID,APPL_01_FLAG,APPL_02_FLAG,APPL_03_FLAG,APPL_04_FLAG,APPL_05_FLAG,APPL_06_FLAG,APPL_07_FLAG,APPL_08_FLAG,LP_DATA_SOURCE,HANDSCANNER_VERSION,ZAHLUNGSART,RUECKFUEHRUNGSENTGELT_BETRAG,UNTERZEICHNER_NAME,MANUELLE_EINGABE,SENDUNGSART_2,PRODUKT_TYP,ZUORDNUNGS_ID,TAS_AUFTRAGS_NR,AUFTRAGSKENNZIFFER,UNBAR_BETRAG,BAR_BETRAG,VERSCHLUESSELUNG,CEP_VERS,CEP_MINLESE_VERS,STATIONSKENNUNG,PST_PLZ,PST_NR,UNTERSCHRIFTSART,SST_PST_NEXTT,SENDUNGS_CODE_TYP,ANZ_N_NACHW_PFL_SDG,LFD_NR_TAS_M_SCANN,MEZ_KUNDENNUMMER,PST_KUNDEN_NR,DROPPOINT_NR,DIENST_KZ,JOURNALSATZNR,AUSWEIS_SONDERFALL,NACHNAHME_BETRAG2,ENTGELT_BETRAG2,BELEGPOS_ENDE,LOAD_ID,N_AUFTRAGSNUMMER,POSITIONSNUMMER,ESR_KENNZIFFER,STATIONSKENNUNG_2,LEITINFO_ALT,SHIPMENT_NR,AUSLIEFER_FLAG,ANZAHL_PS,POSITION_NO,LC_GESCANNT,SCANNER_ID,PERSONAL_NR,GEB_DATUM,ALTERSGRENZE,STP_ID,SCAN_ORT,GRUENE_KARTE,FILIAL_OZ,IDP_KORR,VERTRAGS_IDC,SESSION_ID,ZUSTELL_FLAG,TEZU_DATUM,GID,ZAHLUNGSEMPFAENGER,KONTONUMMER,BLZ,VERWENDUNGSZWECK_1,VERWENDUNGSZWECK_2,FREITEXTWERT,FREITEXT,FREITEXT_KORREKT,STAATSANGEHOERIGKEIT,STAATSANGEHOERIGKEIT_KORREKT,AUSWEISARTID,MELDEANSCHRIFT,VERTRAGSABWICKLUNG,FELDVALIDIERUNGFEHLGESCHLAGEN,AST_ID,POSTCARD_NR,EEA_FLAG,EEA_STRASSE,EEA_HNR,BETRIEBS_NR_KN,UNTERZEICHNER_NAME_KN,EVENT_ID,ARCH_ZEITRAUM,ETL1_DATA_SOURCE_ID,ETL1_LOAD_ID,ETL1_LOAD_DTM,ETL1_LINE_NO,IBAN,BIC,SERVICE_KUERZEL,EVENT_PSLZ_ID,ERZEUGUNGSORT_LAENDERCODE,LAGERN_BIS_DT,UNTERSCHRIFT_BILD_ID,UNTERSCHRIFT_BILD_INHALT,UNTERSCHRIFT_BILD_EXISTS_FLAG,AUTHENTIFIZIERUNGS_TYP,UNTERSCHRIFT_BILD_MIME_TYPE,ERZEUGUNGSORT_AGNR,ARCH_FLAG,ESR_ZUSTELL_DATUM,PAKETKASTEN_ID,POSTMAN_POSTNUMBER,POSTMAN_LOGISTIC_PROVIDER,LOGISTIK_NETZ,PPO_VERS,EINLIEFERER_VORNAME,EINLIEFERER_NAME,EINLIEFERER_STRASSE,EINLIEFERER_HNR,EINLIEFERER_PLZ,EINLIEFERER_ORT,EINLIEFERER_LAENDERCODE,EMPFAENGER_VORNAME,EMPFAENGER_STRASSE,EMPFAENGER_HNR,EMPFAENGER_PLZ,EMPFAENGER_ORTSNAME,EMPFAENGER_LAENDERCODE,EVENT_ART_PAKETSHOP,NICHT_ZUSTELLUNGSGRUND,UEBERGABE_VON,UEBERGABE_ZU,EXTERNE_PAKETSHOP_ID,ISBT_PAKETSHOP_NR,PLZ_CODE_PAKETSHOP_NR,PAKETSHOP_LAENGENGRAD,PAKETSHOP_BREITENGRAD,PAKETSHOP_STRASSEN_NAME,PAKETSHOP_HAUSNUMMER,PAKETSHOP_HAUSNUMMER_ZUSATZ,PAKETSHOP_PLZ,PAKETSHOP_ORTS_NAME,PAKETSHOP_LAENDERCODE,PAKETSHOP_INTERNATIONAL_NR,PAKETSHOP_BETRIEBSMODUS,SERVICE_EMPFAENGER_NACHNAME,SERVICE_EMPFAENGER_VORNAME,PAKET_STATUS,FSHDUPLIKAT_FLAG,GUELTIG_FLAG,ODS_EVENT_ID,PRIMARY_CRITERIA,SECONDARY_CRITERIA,CORR_TO_BE_ARCHIVED_ORIGINATOR,CORR_SCRIPT_ID,CORR_INSTRUCTION_ID,CORR_PROTOCOL_VERSION,CORR_ERROR_CODE,CORR_CHECK_CODE,CORR_ORIG_EVENT_ID,CREATION_LOC_ADDRESS_COUNTRY,PROD_LOCATION_CODE_AGNR,FORMAT_ADJUST_SYNTHETIC_EVENT,FORMAT_ADJUST_CORRECTION_TS,FORMAT_ADJUST_REVISER,FORMAT_ADJUST_COMMENT,FORMAT_ADJUST_POST_PROC_ADJUST,FORMAT_ADJUST_ORIG_EVENT_ID,FORMAT_ADJUST_ORIG_EVT_PRED_ID,EVT_MSG_HDR_DS_DSTID,PSLZ_ID,SHIP_ID,VALID_ROUTING_CODE_TYPE_1,VALID_RC_1_COUNTRY_CODE,VALID_RC_1_POSTAL_CODE,VALID_RC_1_STREET_CODE,VALID_RC_1_HOUSE_NUMBER,VALID_RC_1_PRODUCT_CODE_BASIS,VALID_RC1_PRODUCT_CODE_FEATURE,VALID_ROUTING_CODE_TYPE_2,VALID_RC_2_COUNTRY_CODE,VALID_RC_2_POSTAL_CODE,VALID_RC_2_STREET_CODE,VALID_RC_2_HOUSE_NUMBER,VALID_RC_2_PRODUCT_CODE_BASIS,VALID_RC2_PRODUCT_

# Data Collection

In [2]:
now = date.today()
jahr_akt = now.year
monat_akt = now.month 


if monat_akt == 1:
    monat = 12
    jahr = jahr_akt - 1
else:
    monat = monat_akt - 1
    jahr = jahr_akt


datum1 = date(year = jahr, month = monat, day = 1)  # Erster Tag der Analyse
datum2 = date(year = jahr_akt, month = monat_akt, day = 1)  # Erster Tag der Analyse
#datum2 = datum1 + timedelta(days=-2)
datum1_PAN = datum1 + timedelta(days=-10)

print(datum1, datum2, datum1_PAN)
        
 
    #Identify import parcels:

# •	UPU S10 Sendung_code (Type 6) 
# o	starts with C, do not ends with DE
# o	starts with EP, ends with CH or IT
# o	starts with EE or EK, ends with GB
# o	starts with EH, ends with GB (these are usually shipments from Germany that were undeliverable in GB, but for which the original IDC is unfortunately no longer recognizable)
# o	starts with EG-EO or EQ-ER, ends with IE
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_int_ohne_PAN_0a'
SQL = f'''
        select distinct sendungs_code from
            (
            select       sendungs_code
            from DB_NEXTT.PZA_event
            where   ereignis_datum >= cast('{datum1}' as date)  and ereignis_datum < cast('{datum2}' as date)
                    and LEIT_PRODUKT_CODE = 74
                    and         ((substr(sendungs_code,1,1) = 'C' and right(sendungs_code,2) <> 'DE')  or
                                 (substr(sendungs_code,1,2) = 'EP' and right(sendungs_code,2) ='CH') or
                                 (substr(sendungs_code,1,2) = 'EP' and right(sendungs_code,2) ='IT') or
                                 (substr(sendungs_code,1,2) = 'EE' and right(sendungs_code,2) ='GB') or
                                 (substr(sendungs_code,1,2) = 'EK' and right(sendungs_code,2) ='GB') or
                                 (substr(sendungs_code,1,2) = 'EH' and right(sendungs_code,2) ='GB') or
                                 (substr(sendungs_code,1,5) = 'EG-EO' and right(sendungs_code,2) ='IE') or
                                 (substr(sendungs_code,1,5) = 'EQ-ER' and right(sendungs_code,2) ='IE')) 
                                 
            union all

            select       sendungs_code
            from DB_NEXTT.PZE_event
            where      ereignis_datum >= cast('{datum1}' as date)  and ereignis_datum < cast('{datum2}' as date)
                    and LEIT_PRODUKT_CODE = 74
                    and         ((substr(sendungs_code,1,1) = 'C' and right(sendungs_code,2) <> 'DE')  or
                                 (substr(sendungs_code,1,2) = 'EP' and right(sendungs_code,2) ='CH') or
                                 (substr(sendungs_code,1,2) = 'EP' and right(sendungs_code,2) ='IT') or
                                 (substr(sendungs_code,1,2) = 'EE' and right(sendungs_code,2) ='GB') or
                                 (substr(sendungs_code,1,2) = 'EK' and right(sendungs_code,2) ='GB') or
                                 (substr(sendungs_code,1,2) = 'EH' and right(sendungs_code,2) ='GB') or
                                 (substr(sendungs_code,1,5) = 'EG-EO' and right(sendungs_code,2) ='IE') or
                                 (substr(sendungs_code,1,5) = 'EQ-ER' and right(sendungs_code,2) ='IE')) 
            )    as tmp 
            
        '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)




Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_int_0'
SQL = f'''
        select distinct sendungs_code from
            (
            select       sendungs_code
            from DB_NEXTT.PZA_event
            where      ereignis_datum >= cast('{datum1}' as date)  and ereignis_datum < cast('{datum2}' as date)
                    and sendungs_code not in (select sendungs_code from DBX_DWH_SBX_GB23_PRD.Herkules_int_ohne_PAN_0a)
                    and         ((substr(sendungs_code,1,1) = 'C' and right(sendungs_code,2) <> 'DE')  or
                                 (substr(sendungs_code,1,2) = 'EP' and right(sendungs_code,2) ='CH') or
                                 (substr(sendungs_code,1,2) = 'EP' and right(sendungs_code,2) ='IT') or
                                 (substr(sendungs_code,1,2) = 'EE' and right(sendungs_code,2) ='GB') or
                                 (substr(sendungs_code,1,2) = 'EK' and right(sendungs_code,2) ='GB') or
                                 (substr(sendungs_code,1,2) = 'EH' and right(sendungs_code,2) ='GB') or
                                 (substr(sendungs_code,1,5) = 'EG-EO' and right(sendungs_code,2) ='IE') or
                                 (substr(sendungs_code,1,5) = 'EQ-ER' and right(sendungs_code,2) ='IE'))
                                 
            union all

            select       sendungs_code
            from DB_NEXTT.PZE_event
            where      ereignis_datum >= cast('{datum1}' as date)  and ereignis_datum < cast('{datum2}' as date)
                    and sendungs_code not in (select sendungs_code from DBX_DWH_SBX_GB23_PRD.Herkules_int_ohne_PAN_0a)
                    and         ((substr(sendungs_code,1,1) = 'C' and right(sendungs_code,2) <> 'DE')  or
                                 (substr(sendungs_code,1,2) = 'EP' and right(sendungs_code,2) ='CH') or
                                 (substr(sendungs_code,1,2) = 'EP' and right(sendungs_code,2) ='IT') or
                                 (substr(sendungs_code,1,2) = 'EE' and right(sendungs_code,2) ='GB') or
                                 (substr(sendungs_code,1,2) = 'EK' and right(sendungs_code,2) ='GB') or
                                 (substr(sendungs_code,1,2) = 'EH' and right(sendungs_code,2) ='GB') or
                                 (substr(sendungs_code,1,5) = 'EG-EO' and right(sendungs_code,2) ='IE') or
                                 (substr(sendungs_code,1,5) = 'EQ-ER' and right(sendungs_code,2) ='IE'))
            )    as tmp                 
        '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)


#get the  weight for the import parcels where we do not have the real weighing scale as pan weight from PZA_table
#(we are taking this because we do not have pan data for import parcels)
#also last two digits of sendungs code is country from  where the parcel is sent
    
    
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_int_1'
SQL = f'''

            select 	sendungs_code, ereignis_datum, gewicht as gewicht_pan
            from DB_NEXTT.PZA_event
            where 	ereignis_datum >= cast('{datum1}' as date)  and ereignis_datum < cast('{datum2}' as date)
                            and Gewicht_Quelle not in ('W', 'Y')
                            and gewicht > 0 
                            and sendungs_code in (Select sendungs_code from DBX_DWH_SBX_GB23_PRD.Herkules_int_0)
        

        '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)
    
#get the data from DB_NEXTT.PZA_event, DB_NEXTT.PZE_event, DB_NEXTT_Hist.PZA_event, DB_NEXTT_Hist.PZE_event into single table 
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_int_2'
SQL = f'''
            Select
                sendungs_code,
                right(sendungs_code,2) as country_code,
                ERFASSUNGSORT as PZ_NR,
                min(LEIT_PRODUKT_CODE) as product_code,
                avg(gewicht) as gewicht_real,
                min(gewicht_quelle) as gewicht_quelle,
                avg(volumen_pals) as Volumen,
                avg(laenge) as laenge,
                avg(breite) as breite,
                avg(hoehe) as hoehe,
                count(sendungs_code) as anzahl_messungen,
                max(ereignis_zeitstempel) as pza_timestamp, 
                max(ereignis_datum) as pza_datum,
                min(ereignis_typ) as ereignis_typ, 
                max(zeit) as pza_uhrzeit,
                max(ZIELKENNUNG_AGNR) as ziel_pze_agnr,
                min(WEIGHT_LEGAL_CODE) as Status_PMES_1,
                max(WEIGHT_LEGAL_CODE) as Status_PMES_2
            from (
                    Select 
                        sendungs_code, ERFASSUNGSORT, LEIT_PRODUKT_CODE, gewicht, gewicht_quelle, volumen_pals, laenge, breite, hoehe,
                        ereignis_zeitstempel, ereignis_datum, ereignis_typ, 
                        EXTRACT( HOUR FROM ereignis_zeitstempel)*60+EXTRACT(MINUTE FROM ereignis_zeitstempel) as zeit,
                        cast(ziel_pz as int) as ziel_pz, ZIELKENNUNG_AGNR, WEIGHT_LEGAL_CODE

                    from DB_NEXTT.PZA_event
                    where 	ereignis_datum >= cast('{datum1}' as date)  and ereignis_datum < cast('{datum2}' as date)
                            and LEIT_PRODUKT_CODE not in (72, 73)
                            and Gewicht_Quelle in ('W', 'Y')
                            and gewicht > 0  
                            and sendungs_code in (Select sendungs_code from DBX_DWH_SBX_GB23_PRD.Herkules_int_0)
                            
                    union all

            
                    Select 
                        sendungs_code, ERFASSUNGSORT, LEIT_PRODUKT_CODE, gewicht, gewicht_quelle, volumen_pals, laenge, breite, hoehe,
                        ereignis_zeitstempel, ereignis_datum, ereignis_typ, 
                        EXTRACT( HOUR FROM ereignis_zeitstempel)*60+EXTRACT(MINUTE FROM ereignis_zeitstempel) as zeit,
                        cast(ziel_pz as int) as ziel_pz, ZIELKENNUNG_AGNR, WEIGHT_LEGAL_CODE
                    from DB_NEXTT.PZE_event
                    where 	ereignis_datum >= cast('{datum1}' as date)  and ereignis_datum < cast('{datum2}' as date)
                            and LEIT_PRODUKT_CODE not in (72, 73)
                            and Gewicht_Quelle in ('W', 'Y')
                            and gewicht > 0  
                            and sendungs_code in (Select sendungs_code from DBX_DWH_SBX_GB23_PRD.Herkules_int_0)

                 
        
                    union all
        
                    Select 
                        sendungs_code, ERFASSUNGSORT, LEIT_PRODUKT_CODE, gewicht, gewicht_quelle, volumen_pals, laenge, breite, hoehe,
                        ereignis_zeitstempel, ereignis_datum, ereignis_typ, 
                        EXTRACT( HOUR FROM ereignis_zeitstempel)*60+EXTRACT(MINUTE FROM ereignis_zeitstempel) as zeit,
                        cast(ziel_pz as int) as ziel_pz, ZIELKENNUNG_AGNR, WEIGHT_LEGAL_CODE
                    from DB_NEXTT_Hist.PZA_event
                    where 	ereignis_datum >= cast('{datum1}' as date)  and ereignis_datum < cast('{datum2}' as date)
                            and LEIT_PRODUKT_CODE not in (72, 73)
                            and Gewicht_Quelle in ('W', 'Y')
                            and gewicht > 0  
                            and sendungs_code in (Select sendungs_code from DBX_DWH_SBX_GB23_PRD.Herkules_int_0)
                 

                    union all

            
                    Select 
                        sendungs_code, ERFASSUNGSORT, LEIT_PRODUKT_CODE, gewicht, gewicht_quelle, volumen_pals, laenge, breite, hoehe,
                        ereignis_zeitstempel, ereignis_datum, ereignis_typ, 
                        EXTRACT( HOUR FROM ereignis_zeitstempel)*60+EXTRACT(MINUTE FROM ereignis_zeitstempel) as zeit,
                        cast(ziel_pz as int) as ziel_pz, ZIELKENNUNG_AGNR, WEIGHT_LEGAL_CODE
                    
                    from DB_NEXTT_Hist.PZE_event
                    where 	ereignis_datum >= cast('{datum1}' as date)  and ereignis_datum < cast('{datum2}' as date)
                            and LEIT_PRODUKT_CODE not in (72, 73)
                            and Gewicht_Quelle in ('W', 'Y')
                            and gewicht > 0  
                            and sendungs_code in (Select sendungs_code from DBX_DWH_SBX_GB23_PRD.Herkules_int_0)

            ) as tmp
            group by sendungs_code, ERFASSUNGSORT
        
        '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)
 

    
 #Collecting the shape of the parcel data from VW_PEM_ADD table for the parcels which are in Herkules_Train_int_1 
    
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_int_3'
SQL = f'''
            Select 		piece_code as sendungs_code,
                        special_type_61 as BAP_feature,
                        tb_load_dtm,
                        case 
                        when BAP_feature.SHAPE is NOT NULL then BAP_feature.SHAPE
                        else 'None'
                        end as parcel_shape
                    
                        
            from 		DB_NEXTT_SHIP.VW_PEM_ADD 
            
    
            where 		tb_load_dtm >= cast('{datum1}' as date)  and tb_load_dtm < cast('{datum2}' as date)
                        and sendungs_code in (select sendungs_code from DBX_DWH_SBX_GB23_PRD.Herkules_int_0 )
                        and parcel_shape is not null
            Qualify (ROW_NUMBER() OVER(PARTITION BY sendungs_code ORDER BY tb_load_dtm ASC)) = 1
            

    '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)
    
#Joining all the three tables above and also country data from DB_NEXTT_DWH.vw_pan_partner_recipient table 
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_int_4' 
SQL = f'''

            select distinct 	PZ.sendungs_code as sendungs_code,PZ.PZ_NR, pz.Gewicht_REAL, pz.Gewicht_Quelle,pz.product_code,
                    pz.Volumen, pz.laenge, pz.breite, pz.hoehe,pz.anzahl_messungen,pz.pza_datum,pz.ziel_pze_agnr,pz.Status_PMES_1,pz.Status_PMES_2,
                    PAN.ereignis_datum,PAN.Gewicht_PAN,
                    case 
                    when PZ.country_code in ('AT','BE','DK', 'FI', 'FR', 'NL', 'SE', 'LU', 'PL', 'CZ', 'ES', 'IT', 'SK',
                    'SI', 'HU', 'EE', 'LV', 'LT', 'GB', 'CH', 'IE', 'BG', 
                    'RO', 'CY', 'HR', 'PT', 'GR', 'MT', 'NO', 'US', 'CN', 
                    'MC', 'LI', 'CA', 'JP', 'AU', 'KR', 'UA',  'TW', 
                    'RU', 'TR',  'SM', 'KP', 'BR',  'VA', 'IS', 
                    'AD', 'RS',   'BY',  'NZ',  'KZ',
                    'IN', 'TH') then PZ.country_code
                    else 'XX'
                    end as country_code,
                    case 
                    when BAP.parcel_shape = 'bag' then 'bag' 
                    when BAP.parcel_shape = 'cuboid' then 'cuboid' 
                    when BAP.parcel_shape = 'envelope' then 'envelope'
                    else 'None'
                    end as parcel_shape
            from DBX_DWH_SBX_GB23_PRD.Herkules_int_2 as PZ
            left join DBX_DWH_SBX_GB23_PRD.Herkules_int_1 as PAN
            on 	PZ.sendungs_code = PAN.sendungs_code
            left join DBX_DWH_SBX_GB23_PRD.Herkules_int_3 as BAP
            on PZ.sendungs_code = BAP.sendungs_code
            

        '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)
    
drop_table('DBX_DWH_SBX_GB23_PRD.Herkules_int_1')
drop_table('DBX_DWH_SBX_GB23_PRD.Herkules_int_2')
drop_table('DBX_DWH_SBX_GB23_PRD.Herkules_int_3')
    
#crete new target features and create volumen_grp feature
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_int_CV'
SQL = f'''

    select  A.*, 
                case
                    when Gewicht_Real < 20 then  0
                    when Gewicht_Real >= 20  then 1
                    else 0
               end as Flag_Gewicht_20,
               case
                    when Gewicht_Real < 10 then  0
                    when Gewicht_Real >= 10  and  Gewicht_Real < 20 then 1
                else 0
               
               end as Flag_Gewicht_10,

                case 
                    when volumen < 20 then 'V20'
                    when volumen < 30 then 'V30'
                    when volumen < 40 then 'V40'
                    when volumen < 50 then 'V50'
                    when volumen < 70 then 'V70'
                    when volumen < 90 then 'V90'
                    when volumen >= 90 then 'V99'
                    else '?'
                end as volumen_grp


        from DBX_DWH_SBX_GB23_PRD.Herkules_int_4 as A    
                    
        '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)    
drop_table('DBX_DWH_SBX_GB23_PRD.Herkules_int_4')
    
    



2024-08-01 2024-09-01 2024-07-22
DBX_DWH_SBX_GB23_PRD.Herkules_int_ohne_PAN_0a dropped.
DBX_DWH_SBX_GB23_PRD.Herkules_int_ohne_PAN_0a initialised with PI sendungs_code
10533 DS injected into table DBX_DWH_SBX_GB23_PRD.Herkules_int_ohne_PAN_0a.
 
DBX_DWH_SBX_GB23_PRD.Herkules_int_0 dropped.
DBX_DWH_SBX_GB23_PRD.Herkules_int_0 initialised with PI sendungs_code
296943 DS injected into table DBX_DWH_SBX_GB23_PRD.Herkules_int_0.
 
DBX_DWH_SBX_GB23_PRD.Herkules_int_1 nicht vorhanden
DBX_DWH_SBX_GB23_PRD.Herkules_int_1 initialised with PI sendungs_code
48278 DS injected into table DBX_DWH_SBX_GB23_PRD.Herkules_int_1.
 
DBX_DWH_SBX_GB23_PRD.Herkules_int_2 nicht vorhanden
DBX_DWH_SBX_GB23_PRD.Herkules_int_2 initialised with PI sendungs_code
63345 DS injected into table DBX_DWH_SBX_GB23_PRD.Herkules_int_2.
 
DBX_DWH_SBX_GB23_PRD.Herkules_int_3 nicht vorhanden
DBX_DWH_SBX_GB23_PRD.Herkules_int_3 initialised with PI sendungs_code
296943 DS injected into table DBX_DWH_SBX_GB23_PRD.Herkules_int_3.
 

In [6]:
#display(td_connection.select("select * from DBX_DWH_SBX_GB23_PRD.Herkules_int_CV where parcel_shape ='None'"))
#display(td_connection.select("select count(1) as anzahl,parcel_shape from DBX_DWH_SBX_GB23_PRD.Herkules_int_CV group by 2 "))
#display(td_connection.select("select count(1) as anzahl,gewicht_pan from DBX_DWH_SBX_GB23_PRD.Herkules_int_CV group by 2 order by 2 "))
#display(td_connection.select("select count(1) as anzahl,volumen_grp from DBX_DWH_SBX_GB23_PRD.Herkules_int_CV group by 2 order by 2 "))
#display(td_connection.select("select count(1) as anzahl, country_code from DBX_DWH_SBX_GB23_PRD.Herkules_int_CV group by 2 order by 2 "))


# Now join the leading table with history tables

In [3]:
#Left join the history data features with weighing scale to the main data table
Tabelle = 'DBX_DWH_SBX_GB23_PRD.GS_Herkules_CV_int_Join1' 
SQL = f'''

   SELECT 	A.*,B.Anzahl_GM,B.D_Anzahl_GM,
			B.D_Hoehe_GM, B.D_Laenge_GM, B.D_Breite_GM, 
			B.D_Gewicht_real_GM, B.D_Gewicht_pan_GM, B.D_Volumen_GM,
            B.D_hoehe_min_GM, B.D_hoehe_max_GM, B.D_laenge_min_GM, B.D_laenge_max_GM, B.D_breite_min_GM, B.D_breite_max_GM, 
            B.D_Gewicht_real_min_GM, B.D_Gewicht_real_max_GM, B.D_gewicht_pan_min_GM, B.D_gewicht_pan_max_GM,
            B.D_Volumen_min_GM,B.D_Volumen_max_GM,
            B.D_Volumengewicht_real_GM, B.D_Volumengewicht_pan_GM,
			B.D_LBH_Summe_GM, B.D_Volumengewicht_real2_GM, B.D_Volumengewicht_pan2_GM, 
            B.D_Gewichtsabweichung_GM,B.Anzahl_GU_20, B.Anzahl_GU_10, B.D_Anteil_GU_20, B.D_Anteil_GU_10,
            B.D_laenge_diff_GM, B.D_breite_diff_GM, B.D_hoehe_diff_GM, B.D_Volumen_diff_GM,
            B.D_gewicht_pan_diff_GM, B.D_gewicht_real_diff_GM,B.D_Volumengewicht_diff_GM
   FROM DBX_DWH_SBX_GB23_PRD.Herkules_int_CV as A
   left join DBX_DWH_SBX_GB23_PRD.GS_stammdaten_Herkules_int_GM as B
   on A.country_code = B.country_code
      and A.parcel_shape = b.parcel_shape
      and A.volumen_grp = B.volumen_grp
   
   
    '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)

#test =td_connection.select(f'''select * from {Tabelle} ''')
#test.head(15)

DBX_DWH_SBX_GB23_PRD.GS_Herkules_CV_int_Join1 dropped.
DBX_DWH_SBX_GB23_PRD.GS_Herkules_CV_int_Join1 initialised with PI sendungs_code
63525 DS injected into table DBX_DWH_SBX_GB23_PRD.GS_Herkules_CV_int_Join1.
 


In [4]:
#Left join the history data features with and without weighing scale to the main data table
Tabelle = 'DBX_DWH_SBX_GB23_PRD.GS_Herkules_CV_int_Join2' 
SQL = f'''

   SELECT 	A.*,
			B.Anzahl,B.D_Anzahl,B.D_Hoehe,B.D_Laenge,B.D_Breite,B.D_Gewicht,B.D_Volumen,
            B.D_hoehe_min,B.D_hoehe_max, B.D_laenge_min, B.D_laenge_max, B.D_breite_min, B.D_breite_max, B.D_gewicht_min,
            B.D_gewicht_max, B.D_Volumen_min, B.D_Volumen_max,B.D_LBH_Summe,
            B.D_Volumengewicht_pan,B.D_Volumengewicht_pan2,B.D_Volumengewicht_pan3
   FROM DBX_DWH_SBX_GB23_PRD.GS_Herkules_CV_int_Join1 as A
   left join DBX_DWH_SBX_GB23_PRD.GS_Herkules_stammdaten_OGM as B
   on A.country_code = B.country_code
      and A.parcel_shape = b.parcel_shape
      and A.volumen_grp = B.volumen_grp
   
   
    '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)

#test =td_connection.select(f'''select * from {Tabelle} ''')
#test.head(15)

DBX_DWH_SBX_GB23_PRD.GS_Herkules_CV_int_Join2 dropped.
DBX_DWH_SBX_GB23_PRD.GS_Herkules_CV_int_Join2 initialised with PI sendungs_code
63525 DS injected into table DBX_DWH_SBX_GB23_PRD.GS_Herkules_CV_int_Join2.
 


In [5]:
#Tabelle = 'DBX_DWH_SBX_GB23_PRD.GS_Basisdaten_Int_Scoring_ALT' 
Tabelle = 'DBX_DWH_SBX_GB23_PRD.GS_CV_Herkules_Int_Scoring' 
SQL = f'''

     select  a.*,
        
        case
            when Hoehe is Null or Hoehe = 0 then 0 
            else round(Laenge / Hoehe) 
        end as VH_Laenge_Hoehe,
        case
            when Breite is Null or Breite = 0 then 0 
            else round(Laenge / Breite) 
        end as VH_Laenge_Breite,
        case
            when Breite is Null or Breite = 0 then 0 
            else round(Hoehe / Breite) 
        end as VH_Hoehe_Breite,
        case
            when D_Laenge is Null or D_Laenge = 0 then 0 
            else round(Laenge / D_Laenge) 
        end as VH_Laenge_DLaenge,
       
        case
            when D_Breite is Null or D_Breite = 0 then 0 
            else round(Breite / D_Breite) 
        end as VH_Breite_DBreite,
       
        case
            when D_Hoehe is Null or D_Hoehe = 0 then 0 
            else round(Hoehe / D_Hoehe) 
        end as VH_Hoehe_DHoehe,
       
        case
            when D_Volumen is Null or D_Volumen = 0 then 0 
            else round(Volumen / D_Volumen) 
        end as VH_Volumen_DVolumen,

        case
            when D_Gewicht_real_gm is Null or D_Gewicht_real_gm = 0 then 0 
            else round(Gewicht_PAN / D_Gewicht_real_gm) 
        end as VH_Gewicht_DGewicht,
        case
            when D_Gewicht is Null or D_Gewicht = 0 then 0 
            else least(10,round(Gewicht_PAN / D_Gewicht)) 
        end as VH_Gewicht_DGewicht_PAN,
           
        greatest(-10,least(15,round(Gewicht_PAN - D_Gewicht))) +10 as DGewichtsabweichung3,
        greatest(-10,least(15,round(Gewicht_PAN - D_Gewicht_real_gm))) +10 as DGewichtsabweichung4,
        greatest(-10,least(10,round(D_Gewicht - D_Gewicht_real_gm))) +10 as DGewichtsabweichung5,    
        greatest(-10,least(15,round(Gewicht_PAN - D_Gewicht_pan_gm))) +10 as DGewichtsabweichung6,
        
        greatest(4,least(20,round((Laenge + Breite + Hoehe)/100))) as LBH_Summe,
        greatest(70,least(150,round((LBH_Summe*1.00 / D_LBH_Summe_GM*1.00) * 100))) as LBH_VH_gm,
        
        case when Volumen > 0 then least(50,round(Gewicht_pan / Volumen*100)) else 0 end as Volumengewicht_pan,
        case when Volumen > 0 then least(50,round(D_Gewicht_real_gm / Volumen*100)) else 0 end as Volumengewicht_pan2,
        case when Gewicht_pan > 0 then least(15,round(D_LBH_Summe_GM / Gewicht_pan)) else 0 end as Volumengewicht_pan3
            
        
       
        
   FROM DBX_DWH_SBX_GB23_PRD.GS_Herkules_CV_int_Join2 AS a
   

   
    '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)


#should we remove the where filter Gewicht_Pan <> 0 and Volumen <>0 ?? as the size of dataset greatly decreased using this filter


DBX_DWH_SBX_GB23_PRD.GS_CV_Herkules_Int_Scoring dropped.
DBX_DWH_SBX_GB23_PRD.GS_CV_Herkules_Int_Scoring initialised with PI sendungs_code
63525 DS injected into table DBX_DWH_SBX_GB23_PRD.GS_CV_Herkules_Int_Scoring.
 


In [6]:
Tabelle = 'DBX_DWH_SBX_GB23_PRD.GS_CV_Herkules_Int_Scoring_all' 
SQL = f'''

    select A.*,B.anzahl_land_flag_ratio
    from DBX_DWH_SBX_GB23_PRD.GS_CV_Herkules_Int_Scoring as A
    left join DBX_DWH_SBX_GB23_PRD.GS_Basisdaten_Herkules_land_flag as B
    on A.country_code = B.country_code
 '''
Index = 'sendungs_code'
create_table(Tabelle, SQL, Index)

DBX_DWH_SBX_GB23_PRD.GS_CV_Herkules_Int_Scoring_all dropped.
DBX_DWH_SBX_GB23_PRD.GS_CV_Herkules_Int_Scoring_all initialised with PI sendungs_code
63525 DS injected into table DBX_DWH_SBX_GB23_PRD.GS_CV_Herkules_Int_Scoring_all.
 


In [7]:

# Download the data from Teradata:
import time

# It is important to have a schema with write permissions for the teradata interpreter so that the data types can be converted correctly!
startzeit = time.time()

Used_Sandbox = "DBX_DWH_SBX_GB23_PRD"  
Tabelle = "GS_CV_Herkules_Int_Scoring_all"
target_schema = 'DBX_DWH_SBX_GB23_PRD'

display(td_connection.select(f'''SELECT * FROM {Used_Sandbox}.{Tabelle} sample 5'''))

HK_CVset_int =td_connection.select(f''' SELECT   * FROM {Used_Sandbox}.{Tabelle} ''')
HK_CVset_int.head()

endzeit = time.time()-startzeit

print("Der Import hat ", endzeit , "Sekunden gedauert!")



,sendungs_code,PZ_NR,gewicht_real,gewicht_quelle,product_code,Volumen,laenge,breite,hoehe,anzahl_messungen,pza_datum,ziel_pze_agnr,Status_PMES_1,Status_PMES_2,EREIGNIS_DATUM,gewicht_pan,country_code,parcel_shape,Flag_Gewicht_20,Flag_Gewicht_10,volumen_grp,Anzahl_GM,D_Anzahl_GM,D_Hoehe_GM,D_Laenge_GM,D_Breite_GM,D_Gewicht_real_GM,D_Gewicht_pan_GM,D_Volumen_GM,D_hoehe_min_GM,D_hoehe_max_GM,D_laenge_min_GM,D_laenge_max_GM,D_breite_min_GM,D_breite_max_GM,D_Gewicht_real_min_GM,D_Gewicht_real_max_GM,D_gewicht_pan_min_GM,D_gewicht_pan_max_GM,D_Volumen_min_GM,D_Volumen_max_GM,D_Volumengewicht_real_GM,D_Volumengewicht_pan_GM,D_LBH_Summe_GM,D_Volumengewicht_real2_GM,D_Volumengewicht_pan2_GM,D_Gewichtsabweichung_GM,Anzahl_GU_20,Anzahl_GU_10,D_Anteil_GU_20,D_Anteil_GU_10,D_laenge_diff_GM,D_breite_diff_GM,D_hoehe_diff_GM,D_Volumen_diff_GM,D_gewicht_pan_diff_GM,D_gewicht_real_diff_GM,D_Volumengewicht_diff_GM,Anzahl,D_Anzahl,D_Hoehe,D_Laenge,D_Breite,D_Gewicht,D_Volumen,D_hoehe_min,D_hoehe_max,D_laenge_min,D_laenge_max,D_breite_min,D_breite_max,D_gewicht_min,D_gewicht_max,D_Volumen_min,D_Volumen_max,D_LBH_Summe,D_Volumengewicht_pan,D_Volumengewicht_pan2,D_Volumengewicht_pan3,VH_Laenge_Hoehe,VH_Laenge_Breite,VH_Hoehe_Breite,VH_Laenge_DLaenge,VH_Breite_DBreite,VH_Hoehe_DHoehe,VH_Volumen_DVolumen,VH_Gewicht_DGewicht,VH_Gewicht_DGewicht_PAN,DGewichtsabweichung3,DGewichtsabweichung4,DGewichtsabweichung5,DGewichtsabweichung6,LBH_Summe,LBH_VH_gm,Volumengewicht_pan,Volumengewicht_pan2,Volumengewicht_pan3,anzahl_land_flag_ratio
0,CA310723545AT,20,0.65,W,0,6.0,383.0,235.0,72.0,1,2024-08-06,22.11.009.RE,LFT,LFT,None,None,AT,None,0,0,V20,224463,873,102.0,304.0,220.0,1.0,1.0,7.0,0.0,490.0,0.0,1284.0,0.0,808.0,0.02,32.10,0.001,9.999,0.0,19.666667,14.0,14.0,6.0,6.0,6.0,0.0,18,496,0.00,0.00,1284.0,808.0,490.0,19.666667,9.998,32.08,0.0,626110,1715,106.0,306.0,221.0,1.0,8.0,0.0,612.0,0.0,6128.0,0.0,863.0,0.001,230.0,0.0,19.8,6.0,17.0,6.0,0.0,5.0,2.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,10.0,NaN,7.0,117.0,NaN,17.0,0.0,0.0000
1,CP915961640ES,40,0.55,W,0,6.0,306.0,235.0,87.0,1,2024-08-10,49.124.071.BEZG,LFT,LFT,2024-08-09,0.580,ES,None,0,0,V20,15052,68,97.0,321.0,237.0,1.0,1.0,8.0,0.0,508.0,0.0,1221.0,0.0,641.0,0.05,23.60,0.001,9.970,0.0,19.500000,13.0,13.0,7.0,7.0,7.0,0.0,2,58,0.00,0.00,1221.0,641.0,508.0,19.500000,9.969,23.55,0.0,95355,261,97.0,323.0,231.0,1.0,8.0,0.0,353.0,0.0,4149.0,0.0,768.0,0.001,30.0,0.0,19.5,7.0,18.0,7.0,0.0,4.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,10.0,10.0,10.0,6.0,86.0,10.0,17.0,12.0,0.0100
2,CE400213050HR,90,3.15,W,0,11.0,360.0,223.0,138.0,1,2024-08-12,73.1.1.PZ,NLFT,NLFT,None,None,HR,None,0,0,V20,40479,197,128.0,329.0,243.0,2.0,2.0,10.0,0.0,350.0,0.0,1210.0,0.0,643.0,0.05,28.15,0.113,9.980,0.0,19.500000,20.0,20.0,7.0,4.0,4.0,0.0,7,117,0.00,0.00,1210.0,643.0,350.0,19.500000,9.867,28.10,0.0,24629,67,124.0,325.0,239.0,2.0,10.0,0.0,300.0,0.0,1203.0,0.0,658.0,0.001,30.0,0.0,19.5,7.0,22.0,4.0,0.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,10.0,NaN,7.0,100.0,NaN,18.0,0.0,0.0100
3,CW376717151FR,60,1.30,W,0,24.0,349.0,320.0,216.0,1,2024-08-08,93.1.1.PZ,LFT,LFT,2024-08-08,1.300,FR,None,0,0,V30,14031,68,196.0,433.0,299.0,3.0,2.0,24.0,38.0,516.0,68.0,1219.0,110.0,1215.0,0.05,26.40,0.001,9.915,20.0,29.666667,13.0,8.0,9.0,3.0,5.0,1.0,5,188,0.00,1.00,1151.0,1105.0,478.0,9.666667,9.914,26.35,5.0,38650,105,193.0,434.0,298.0,3.0,24.0,38.0,360.0,227.0,1319.0,126.0,653.0,0.001,31.5,20.0,29.5,9.0,10.0,3.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,8.0,8.0,10.0,9.0,9.0,100.0,5.0,13.0,7.0,0.0000
4,CN747942765LT,60,5.85,W,0,18.0,338.0,268.0,202.0,1,2024-08-08,60.13.025.RE,LFT,LFT,None,None,LT,None,0,0,V20,3380,17,131.0,331.0,245.0,2.0,3.0,11.0,0.0,310.0,0.0,1091.0,0.0,750.0,0.05,23.65,0.022,9.840,0.0,19.500000,18.0,27.0,7.0,4.0,2.0,-1.0,1,6,0.00,0.00,1091.0,750.0,310.0,19.500000,9.818,23.60,-9.0,7896,21,132.0,350.0,253.0,3.0,11.0,0.0,334.0,0.0,1407.0,0.0,870.0,0.005,27.1,0.0,19.5,7.0,25.0,2.0,0.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,11.0,NaN,8.0,114.0,

Der Import hat  12.334805965423584 Sekunden gedauert!


In [8]:
HK_CVset_int.shape

(63525, 98)

In [9]:
# Alle Spaltennamen in Kleinbustaben:
HK_CVset_int.columns = map(str.lower, HK_CVset_int.columns)

In [10]:

#Erstelle Backup um das Score-Ergebnis wieder an die orginaldatan anzuspielen:
HK_CVset_int_Backup = HK_CVset_int.copy()

# Data cleaning

In [11]:
drop_col0 = ['ereignis_datum', 'pza_datum','status_pmes_1','status_pmes_2',
            'gewicht_quelle', 'sendungs_code',  'zeitraum_tage', 'product_code',
            'ziel_pze_agnr', 'pz_nr']


HK_CVset_int.drop(drop_col0, axis=1, inplace=True, errors='ignore')

In [12]:
HK_CVset_int.shape

(63525, 89)

In [13]:
# Count the number of NaN values
print('Number of NULL-Values in the Dataset: ' + str(HK_CVset_int.isnull().sum().sum()))

#Spalten mit NULL-Values:
HK_CVset_int.isnull().sum()

Number of NULL-Values in the Dataset: 389921


gewicht_real                     0
volumen                          0
laenge                           0
breite                           0
hoehe                            0
anzahl_messungen                 0
gewicht_pan                  55489
country_code                     0
parcel_shape                     0
flag_gewicht_20                  0
flag_gewicht_10                  0
volumen_grp                      0
anzahl_gm                        2
d_anzahl_gm                      2
d_hoehe_gm                       2
d_laenge_gm                      2
d_breite_gm                      2
d_gewicht_real_gm                2
d_gewicht_pan_gm               239
d_volumen_gm                     2
d_hoehe_min_gm                   2
d_hoehe_max_gm                   2
d_laenge_min_gm                  2
d_laenge_max_gm                  2
d_breite_min_gm                  2
d_breite_max_gm                  2
d_gewicht_real_min_gm            2
d_gewicht_real_max_gm            2
d_gewicht_pan_min_gm

In [14]:

HK_CVset_int['gewicht_pan'] = HK_CVset_int['gewicht_pan'].fillna(0)
HK_CVset_int['anzahl_gm'] = HK_CVset_int['anzahl_gm'].fillna(0)
HK_CVset_int['d_anzahl_gm'] = HK_CVset_int['d_anzahl_gm'].fillna(0)
HK_CVset_int['d_hoehe_gm'] = HK_CVset_int['d_hoehe_gm'].fillna(0)
HK_CVset_int['d_laenge_gm'] = HK_CVset_int['d_laenge_gm'].fillna(0)
HK_CVset_int['d_breite_gm'] = HK_CVset_int['d_breite_gm'].fillna(0)
HK_CVset_int['d_gewicht_real_gm'] = HK_CVset_int['d_gewicht_real_gm'].fillna(0)
HK_CVset_int['d_gewicht_pan_gm'] = HK_CVset_int['d_gewicht_pan_gm'].fillna(0)
HK_CVset_int['d_volumen_gm'] = HK_CVset_int['d_volumen_gm'].fillna(0)
HK_CVset_int['d_hoehe_min_gm'] = HK_CVset_int['d_hoehe_min_gm'].fillna(0)
HK_CVset_int['d_hoehe_max_gm'] = HK_CVset_int['d_hoehe_max_gm'].fillna(0)
HK_CVset_int['d_laenge_min_gm'] = HK_CVset_int['d_laenge_min_gm'].fillna(0)
HK_CVset_int['d_laenge_max_gm'] = HK_CVset_int['d_laenge_max_gm'].fillna(0)
HK_CVset_int['d_breite_min_gm'] = HK_CVset_int['d_breite_min_gm'].fillna(0)
HK_CVset_int['d_breite_max_gm'] = HK_CVset_int['d_breite_max_gm'].fillna(0)
HK_CVset_int['d_gewicht_real_min_gm'] = HK_CVset_int['d_gewicht_real_min_gm'].fillna(0)
HK_CVset_int['d_gewicht_real_max_gm'] = HK_CVset_int['d_gewicht_real_max_gm'].fillna(0)
HK_CVset_int['d_gewicht_pan_min_gm'] = HK_CVset_int['d_gewicht_pan_min_gm'].fillna(0)
HK_CVset_int['d_gewicht_pan_max_gm'] = HK_CVset_int['d_gewicht_pan_max_gm'].fillna(0)
HK_CVset_int['d_volumen_min_gm'] = HK_CVset_int['d_volumen_min_gm'].fillna(0)
HK_CVset_int['d_volumen_max_gm'] = HK_CVset_int['d_volumen_max_gm'].fillna(0)
HK_CVset_int['d_volumengewicht_real_gm'] = HK_CVset_int['d_volumengewicht_real_gm'].fillna(0)
HK_CVset_int['d_volumengewicht_pan_gm'] = HK_CVset_int['d_volumengewicht_pan_gm'].fillna(0)
HK_CVset_int['d_lbh_summe_gm'] = HK_CVset_int['d_lbh_summe_gm'].fillna(0)
HK_CVset_int['d_volumengewicht_real2_gm'] = HK_CVset_int['d_volumengewicht_real2_gm'].fillna(0)
HK_CVset_int['d_volumengewicht_pan2_gm'] = HK_CVset_int['d_volumengewicht_pan2_gm'].fillna(0)
HK_CVset_int['d_gewichtsabweichung_gm'] = HK_CVset_int['d_gewichtsabweichung_gm'].fillna(0)
HK_CVset_int['anzahl_gu_20'] = HK_CVset_int['anzahl_gu_20'].fillna(0)
HK_CVset_int['anzahl_gu_10'] = HK_CVset_int['anzahl_gu_10'].fillna(0)
HK_CVset_int['d_anteil_gu_20'] = HK_CVset_int['d_anteil_gu_20'].fillna(0)
HK_CVset_int['d_anteil_gu_10'] = HK_CVset_int['d_anteil_gu_10'].fillna(0)
HK_CVset_int['anzahl'] = HK_CVset_int['anzahl'].fillna(0)
HK_CVset_int['d_anzahl'] = HK_CVset_int['d_anzahl'].fillna(0)
HK_CVset_int['d_hoehe'] = HK_CVset_int['d_hoehe'].fillna(0)
HK_CVset_int['d_breite'] = HK_CVset_int['d_breite'].fillna(0)
HK_CVset_int['d_laenge'] = HK_CVset_int['d_laenge'].fillna(0)
HK_CVset_int['d_gewicht'] = HK_CVset_int['d_gewicht'].fillna(0)
HK_CVset_int['d_volumen'] = HK_CVset_int['d_volumen'].fillna(0)
HK_CVset_int['d_hoehe_min'] = HK_CVset_int['d_hoehe_min'].fillna(0)
HK_CVset_int['d_hoehe_max'] = HK_CVset_int['d_hoehe_max'].fillna(0)
HK_CVset_int['d_laenge_min'] = HK_CVset_int['d_laenge_min'].fillna(0)
HK_CVset_int['d_laenge_max'] = HK_CVset_int['d_laenge_max'].fillna(0)
HK_CVset_int['d_breite_min'] = HK_CVset_int['d_breite_min'].fillna(0)
HK_CVset_int['d_breite_max'] = HK_CVset_int['d_breite_max'].fillna(0)
HK_CVset_int['d_gewicht_min'] = HK_CVset_int['d_gewicht_min'].fillna(0)
HK_CVset_int['d_gewicht_max'] = HK_CVset_int['d_gewicht_max'].fillna(0)
HK_CVset_int['d_volumen_min'] = HK_CVset_int['d_volumen_min'].fillna(0)
HK_CVset_int['d_volumen_max'] = HK_CVset_int['d_volumen_max'].fillna(0)
HK_CVset_int['d_lbh_summe'] = HK_CVset_int['d_lbh_summe'].fillna(0)
HK_CVset_int['d_volumengewicht_pan'] = HK_CVset_int['d_volumengewicht_pan'].fillna(0)
HK_CVset_int['d_volumengewicht_pan2'] = HK_CVset_int['d_volumengewicht_pan2'].fillna(0)
HK_CVset_int['d_volumengewicht_pan3'] = HK_CVset_int['d_volumengewicht_pan3'].fillna(0)

HK_CVset_int['vh_gewicht_dgewicht'] = HK_CVset_int['vh_gewicht_dgewicht'].fillna(0)
HK_CVset_int['vh_gewicht_dgewicht_pan'] = HK_CVset_int['vh_gewicht_dgewicht_pan'].fillna(0)
HK_CVset_int['dgewichtsabweichung3'] = HK_CVset_int['dgewichtsabweichung3'].fillna(0)
HK_CVset_int['dgewichtsabweichung4'] = HK_CVset_int['dgewichtsabweichung4'].fillna(0)
HK_CVset_int['dgewichtsabweichung5'] = HK_CVset_int['dgewichtsabweichung5'].fillna(0)

HK_CVset_int['dgewichtsabweichung6'] = HK_CVset_int['dgewichtsabweichung6'].fillna(0)
HK_CVset_int['lbh_vh_gm'] = HK_CVset_int['lbh_vh_gm'].fillna(0)
HK_CVset_int['volumengewicht_pan'] = HK_CVset_int['volumengewicht_pan'].fillna(0)
HK_CVset_int['volumengewicht_pan2'] = HK_CVset_int['volumengewicht_pan2'].fillna(0)
HK_CVset_int['volumengewicht_pan3'] = HK_CVset_int['volumengewicht_pan3'].fillna(0)

HK_CVset_int['d_laenge_diff_gm'] = HK_CVset_int['d_laenge_diff_gm'].fillna(0)
HK_CVset_int['d_breite_diff_gm'] = HK_CVset_int['d_breite_diff_gm'].fillna(0)
HK_CVset_int['d_hoehe_diff_gm'] = HK_CVset_int['d_hoehe_diff_gm'].fillna(0)
HK_CVset_int['d_volumen_diff_gm'] = HK_CVset_int['d_volumen_diff_gm'].fillna(0)
HK_CVset_int['d_gewicht_pan_diff_gm'] = HK_CVset_int['d_gewicht_pan_diff_gm'].fillna(0)
HK_CVset_int['d_gewicht_real_diff_gm'] = HK_CVset_int['d_gewicht_real_diff_gm'].fillna(0)
HK_CVset_int['d_volumengewicht_diff_gm'] = HK_CVset_int['d_volumengewicht_diff_gm'].fillna(0)


In [15]:
# Count the number of NaN values
print('Number of NULL-Values in the Dataset: ' + str(HK_CVset_int.isnull().sum().sum()))

#Spalten mit NULL-Values:
HK_CVset_int.isnull().sum()

Number of NULL-Values in the Dataset: 0


gewicht_real                 0
volumen                      0
laenge                       0
breite                       0
hoehe                        0
anzahl_messungen             0
gewicht_pan                  0
country_code                 0
parcel_shape                 0
flag_gewicht_20              0
flag_gewicht_10              0
volumen_grp                  0
anzahl_gm                    0
d_anzahl_gm                  0
d_hoehe_gm                   0
d_laenge_gm                  0
d_breite_gm                  0
d_gewicht_real_gm            0
d_gewicht_pan_gm             0
d_volumen_gm                 0
d_hoehe_min_gm               0
d_hoehe_max_gm               0
d_laenge_min_gm              0
d_laenge_max_gm              0
d_breite_min_gm              0
d_breite_max_gm              0
d_gewicht_real_min_gm        0
d_gewicht_real_max_gm        0
d_gewicht_pan_min_gm         0
d_gewicht_pan_max_gm         0
d_volumen_min_gm             0
d_volumen_max_gm             0
d_volume

In [16]:

DS_one_hot = HK_CVset_int[['volumen_grp', 'parcel_shape','country_code']]
    
# Implement one-hot-encoding with get_dummies:
DS_one_hot2 = DS_one_hot.applymap(str)
DS_one_hot3 = pd.get_dummies(DS_one_hot2)

HK_CVset_int.drop(['volumen_grp', 'parcel_shape','country_code'], axis=1, inplace=True)

HK_CVset_int = HK_CVset_int.join(DS_one_hot3)

HK_CVset_int.head()

/tmp/ipykernel_3871/754529355.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  DS_one_hot2 = DS_one_hot.applymap(str)


,gewicht_real,volumen,laenge,breite,hoehe,anzahl_messungen,gewicht_pan,flag_gewicht_20,flag_gewicht_10,anzahl_gm,d_anzahl_gm,d_hoehe_gm,d_laenge_gm,d_breite_gm,d_gewicht_real_gm,d_gewicht_pan_gm,d_volumen_gm,d_hoehe_min_gm,d_hoehe_max_gm,d_laenge_min_gm,d_laenge_max_gm,d_breite_min_gm,d_breite_max_gm,d_gewicht_real_min_gm,d_gewicht_real_max_gm,d_gewicht_pan_min_gm,d_gewicht_pan_max_gm,d_volumen_min_gm,d_volumen_max_gm,d_volumengewicht_real_gm,d_volumengewicht_pan_gm,d_lbh_summe_gm,d_volumengewicht_real2_gm,d_volumengewicht_pan2_gm,d_gewichtsabweichung_gm,anzahl_gu_20,anzahl_gu_10,d_anteil_gu_20,d_anteil_gu_10,d_laenge_diff_gm,d_breite_diff_gm,d_hoehe_diff_gm,d_volumen_diff_gm,d_gewicht_pan_diff_gm,d_gewicht_real_diff_gm,d_volumengewicht_diff_gm,anzahl,d_anzahl,d_hoehe,d_laenge,d_breite,d_gewicht,d_volumen,d_hoehe_min,d_hoehe_max,d_laenge_min,d_laenge_max,d_breite_min,d_breite_max,d_gewicht_min,d_gewicht_max,d_volumen_min,d_volumen_max,d_lbh_summe,d_volumengewicht_pan,d_volumengewicht_pan2,d_volumengewicht_pan3,vh_laenge_hoehe,vh_laenge_breite,vh_hoehe_breite,vh_laenge_dlaenge,vh_breite_dbreite,vh_hoehe_dhoehe,vh_volumen_dvolumen,vh_gewicht_dgewicht,vh_gewicht_dgewicht_pan,dgewichtsabweichung3,dgewichtsabweichung4,dgewichtsabweichung5,dgewichtsabweichung6,lbh_summe,lbh_vh_gm,volumengewicht_pan,volumengewicht_pan2,volumengewicht_pan3,anzahl_land_flag_ratio,volumen_grp_V20,volumen_grp_V30,volumen_grp_V40,volumen_grp_V50,volumen_grp_V70,volumen_grp_V90,volumen_grp_V99,parcel_shape_None,parcel_shape_bag,parcel_shape_cuboid,parcel_shape_envelope,country_code_AT,country_code_AU,country_code_BE,country_code_BG,country_code_BR,country_code_BY,country_code_CA,country_code_CH,country_code_CN,country_code_CY,country_code_CZ,country_code_DK,country_code_EE,country_code_ES,country_code_FI,country_code_FR,country_code_GB,country_code_GR,country_code_HR,country_code_HU,country_code_IE,country_code_IN,country_code_IS,country_code_IT,country_code_JP,country_code_KR,country_code_KZ,country_code_LI,country_code_LT,country_code_LU,country_code_LV,country_code_MT,country_code_NL,country_code_NO,country_code_NZ,country_code_PL,country_code_PT,country_code_RO,country_code_RS,country_code_RU,country_code_SE,country_code_SI,country_code_SK,country_code_TH,country_code_TR,country_code_TW,country_code_UA,country_code_US,country_code_XX
0,0.50,9.0,407.0,270.0,86.0,1,0,0,0,224463.0,873.0,102.0,304.0,220.0,1.0,1.0,7.0,0.0,490.0,0.0,1284.0,0.0,808.0,0.02,32.10,0.001,9.999,0.0,19.666667,14.0,14.0,6.0,6.0,6.0,0.0,18.0,496.0,0.00,0.00,1284.0,808.0,490.0,19.666667,9.998,32.08,0.0,626110.0,1715.0,106.0,306.0,221.0,1.0,8.0,0.0,612.0,0.0,6128.0,0.0,863.0,0.001,230.00,0.0,19.800000,6.0,17.0,6.0,0.0,5.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,10.0,0.0,8.0,133.0,0.0,11.0,0.0,0.0000,True,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,0.35,3.0,235.0,173.0,83.0,1,0,0,0,159092.0,668.0,111.0,287.0,207.0,1.0,1.0,7.0,0.0,412.0,0.0,1267.0,0.0,840.0,0.02,27.50,0.001,9.430,0.0,19.666667,14.0,14.0,6.0,6.0,6.0,0.0,7.0,447.0,0.00,0.00,1267.0,840.0,412.0,19.666667,9.429,27.48,0.0,419123.0,1148.0,106.0,285.0,206.0,1.0,6.0,0.0,534.0,0.0,1820.0,0.0,887.0,0.001,31.50,0.0,19.666667,6.0,14.0,6.0,0.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,10.0,0.0,5.0,83.0,0.0,33.0,0.0,0.0000,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2.45,13.0,769.5,132.0,131.0,2,2.440,0,0,8505.0,36.0,123

In [17]:
print(HK_CVset_int.shape)
print(DS_one_hot.shape)
print(DS_one_hot2.shape)
print(DS_one_hot3.shape)
#print(GS_Trainset_sample2.shape)
#print(GS_Trainset_sample3.shape)

(63525, 146)
(63525, 3)
(63525, 3)
(63525, 60)


In [18]:
# Count the number of NaN values :
print('Number of NULL-Values in the Dataset: ' + str(HK_CVset_int.isnull().sum().sum()))



Number of NULL-Values in the Dataset: 0


In [19]:
#new data set for SVM model as this need to be normalized
HK_CVset_int_sample_svm = HK_CVset_int.sample(frac=0.99, replace=False, random_state=1)
print(HK_CVset_int_sample_svm.shape)

(62890, 146)


In [20]:
# dropping target variables 
drop_col = ['gewicht_real',  'flag_gewicht_20', 'flag_gewicht_10']

HK_CVset_int.drop(drop_col, axis=1, inplace=True, errors='ignore')

print(HK_CVset_int.shape)

(63525, 143)


In [21]:
# Model laden                                                                                                                                                                                                          

import pickle
Dateiname = 'GS_RF_Klassifikation_HER_GU_20_int.pkl'
print(Dateiname)
Speicherort = '/home/jovyan/shared-data/models_herkules/'+ Dateiname                                                                                                                                                                                         

with open(Speicherort, 'rb') as f:
    RF_Klassifikation_GU_20 = pickle.load(f)

GS_RF_Klassifikation_HER_GU_20_int.pkl


In [22]:
pred_PGU_20 = RF_Klassifikation_GU_20.predict(HK_CVset_int)

In [23]:
#Propability der Prognose ermitteln
p_PGU_20= RF_Klassifikation_GU_20.predict_proba(HK_CVset_int)

In [24]:
# Model laden                                                                                                                                                                                                          

import pickle
Dateiname = 'GS_RF_Prognose_Gewicht_Herkules_int.pkl'
print(Dateiname)
Speicherort = '/home/jovyan/shared-data/models_herkules/'+ Dateiname                                                                                                                                                                                        

with open(Speicherort, 'rb') as f:
    RF_Prediction_GU_20 = pickle.load(f)

GS_RF_Prognose_Gewicht_Herkules_int.pkl


In [25]:
pred_weight_20 = RF_Prediction_GU_20.predict(HK_CVset_int)

In [26]:
# # scaling the data
# from sklearn.preprocessing import StandardScaler

# columns_scaled = HK_CVset_int.columns
# scaler = StandardScaler()

# HK_validationset_scaled = pd.DataFrame(data = HK_CVset_int)
# HK_validationset_scaled[columns_scaled]=scaler.fit_transform(HK_CVset_int[columns_scaled])

# # Show an example of a record with scaling applied
# display(HK_validationset_scaled.head(n = 5))

In [27]:
# # Model laden                                                                                                                                                                                                          

# import pickle
# Dateiname = 'GS_SVM_Klassifikation_HER_GU_20_int.pkl'
# print(Dateiname)
# Speicherort = '/home/jovyan/shared-data/models_herkules/'+ Dateiname                                                                                                                                                                                         

# with open(Speicherort, 'rb') as f:
#     SVM_Klassifikation_GU_20 = pickle.load(f)

In [28]:
# #Prognose erstellen
# pred_PGU_20_SVM = SVM_Klassifikation_GU_20.predict(HK_validationset_scaled)

# #3 min

In [29]:
# #Propability der Prognose ermitteln
# p_PGU_20_SVM = SVM_Klassifikation_GU_20.predict_proba(HK_validationset_scaled)

In [30]:
#p_PGU_20_SVM

# join the predicted outputs to the dataframe

In [31]:
HK_CVset_int['pred_PGU_20'] = pred_PGU_20

In [32]:
HK_CVset_int['p_PGU_20'] = p_PGU_20[:,1]

In [33]:
HK_CVset_int['pred_weight_20'] = pred_weight_20

In [34]:
#HK_CVset_int['pred_PGU_20_SVM'] = pred_PGU_20_SVM

In [35]:
#HK_CVset_int['p_PGU_20_SVM'] = p_PGU_20_SVM[:,1]

In [36]:
results = HK_CVset_int[['pred_PGU_20', 'p_PGU_20','pred_weight_20']]
results.head()

,pred_PGU_20,p_PGU_20,pred_weight_20
0,0,0.0,1.125576
1,0,0.0,0.769074
2,0,0.0,2.178725
3,0,0.0,1.875646
4,0,0.0,2.865466


In [37]:

# Innerjoin:
# Doppler_Scoreergebnis2 = pd.merge(Doppler_Summary_Backup,Doppler_Summary_final[['preds']],left_index = True, right_index = True)

#Leftjoin:
Gewicht_Scoreergebnis = pd.concat([HK_CVset_int_Backup,results],axis=1)
print(len(Gewicht_Scoreergebnis))

Gewicht_Scoreergebnis.head()

63525


,sendungs_code,pz_nr,gewicht_real,gewicht_quelle,product_code,volumen,laenge,breite,hoehe,anzahl_messungen,pza_datum,ziel_pze_agnr,status_pmes_1,status_pmes_2,ereignis_datum,gewicht_pan,country_code,parcel_shape,flag_gewicht_20,flag_gewicht_10,volumen_grp,anzahl_gm,d_anzahl_gm,d_hoehe_gm,d_laenge_gm,d_breite_gm,d_gewicht_real_gm,d_gewicht_pan_gm,d_volumen_gm,d_hoehe_min_gm,d_hoehe_max_gm,d_laenge_min_gm,d_laenge_max_gm,d_breite_min_gm,d_breite_max_gm,d_gewicht_real_min_gm,d_gewicht_real_max_gm,d_gewicht_pan_min_gm,d_gewicht_pan_max_gm,d_volumen_min_gm,d_volumen_max_gm,d_volumengewicht_real_gm,d_volumengewicht_pan_gm,d_lbh_summe_gm,d_volumengewicht_real2_gm,d_volumengewicht_pan2_gm,d_gewichtsabweichung_gm,anzahl_gu_20,anzahl_gu_10,d_anteil_gu_20,d_anteil_gu_10,d_laenge_diff_gm,d_breite_diff_gm,d_hoehe_diff_gm,d_volumen_diff_gm,d_gewicht_pan_diff_gm,d_gewicht_real_diff_gm,d_volumengewicht_diff_gm,anzahl,d_anzahl,d_hoehe,d_laenge,d_breite,d_gewicht,d_volumen,d_hoehe_min,d_hoehe_max,d_laenge_min,d_laenge_max,d_breite_min,d_breite_max,d_gewicht_min,d_gewicht_max,d_volumen_min,d_volumen_max,d_lbh_summe,d_volumengewicht_pan,d_volumengewicht_pan2,d_volumengewicht_pan3,vh_laenge_hoehe,vh_laenge_breite,vh_hoehe_breite,vh_laenge_dlaenge,vh_breite_dbreite,vh_hoehe_dhoehe,vh_volumen_dvolumen,vh_gewicht_dgewicht,vh_gewicht_dgewicht_pan,dgewichtsabweichung3,dgewichtsabweichung4,dgewichtsabweichung5,dgewichtsabweichung6,lbh_summe,lbh_vh_gm,volumengewicht_pan,volumengewicht_pan2,volumengewicht_pan3,anzahl_land_flag_ratio,pred_PGU_20,p_PGU_20,pred_weight_20
0,CA485224047AT,60.0,0.50,W,0,9.0,407.0,270.0,86.0,1,2024-08-09,63.1.1.PZ,LFT,LFT,None,None,AT,None,0,0,V20,224463.0,873.0,102.0,304.0,220.0,1.0,1.0,7.0,0.0,490.0,0.0,1284.0,0.0,808.0,0.02,32.10,0.001,9.999,0.0,19.666667,14.0,14.0,6.0,6.0,6.0,0.0,18.0,496.0,0.00,0.00,1284.0,808.0,490.0,19.666667,9.998,32.08,0.0,626110.0,1715.0,106.0,306.0,221.0,1.0,8.0,0.0,612.0,0.0,6128.0,0.0,863.0,0.001,230.00,0.0,19.800000,6.0,17.0,6.0,0.0,5.0,2.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,10.0,NaN,8.0,133.0,NaN,11.0,0.0,0.0000,0,0.0,1.125576
1,CC068797645FR,43.0,0.35,W,0,3.0,235.0,173.0,83.0,1,2024-08-27,42.16.023.RE,LFT,LFT,None,None,FR,None,0,0,V20,159092.0,668.0,111.0,287.0,207.0,1.0,1.0,7.0,0.0,412.0,0.0,1267.0,0.0,840.0,0.02,27.50,0.001,9.430,0.0,19.666667,14.0,14.0,6.0,6.0,6.0,0.0,7.0,447.0,0.00,0.00,1267.0,840.0,412.0,19.666667,9.429,27.48,0.0,419123.0,1148.0,106.0,285.0,206.0,1.0,6.0,0.0,534.0,0.0,1820.0,0.0,887.0,0.001,31.50,0.0,19.666667,6.0,14.0,6.0,0.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,10.0,NaN,5.0,83.0,NaN,33.0,0.0,0.0000,0,0.0,0.769074
2,CM002214159NL,90.0,2.45,W,0,13.0,769.5,132.0,131.0,2,2024-08-27,90.13.070.RE,NLFT,NLFT,2024-08-24,2.440,NL,None,0,0,V20,8505.0,36.0,123.0,387.0,237.0,2.0,2.0,11.0,0.0,320.0,0.0,1254.0,0.0,945.0,0.05,24.00,0.005,9.896,0.0,19.750000,18.0,18.0,7.0,4.0,4.0,0.0,6.0,37.0,0.00,0.00,1254.0,945.0,320.0,19.750000,9.891,23.95,0.0,43420.0,118.0,111.0,373.0,226.0,3.0,10.0,0.0,588.0,0.0,6656.0,0.0,880.0,0.001,30.34,0.0,19.500000,7.0,26.0,2.0,0.0,6.0,6.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,9.0,10.0,11.0,10.0,10.0,143.0,19.0,15.0,3.0,0.0100,0,0.0,2.178725
3,CP297144822PL,43.0,1.30,W,0,8.0,266.0,214.0,140.0,1,2024-08-16,45.17.027.RE,LFT,LFT,None,None,PL,None,0,0,V20,19009.0,95.0,122.0,319.0,236.0,2.0,2.0,9.0,0.0,325.0,0.0,1136.0,0.0,771.0,0.05,24.10,0.010,9.980,0.0,19.600000,22.0,22.0,7.0,4.0,4.0,0.0,2.0,43.0,0.00,0.00,1136.0,771.0,325.0,19.600000,9.970,24.05,0.0,81351.0,222.0,119.0,317.0,230.0,2.0,9.0,0.0,347.0,0.0,1223.0,0.0,853.0,0.010,23.00,0.0,19.666667,7.0,20.0,4.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,10.0,NaN,6.0,86.0,NaN,25.0,0.0,0.0000,0,0.0,1.875646
4,CC642097749SE,25.0,3.20,W,0,14.0,373.0,351.0,144.0,1,2024-08-16,24.130.058.BEZG,LFT,LFT,None,None,SE,None,0,0,V20,2583.0,47.0,129.0,338.0,250.0,2.0,2.0,11.0,0.0,295.0,0.0,1174.0,0.0,1110.0,0.05,16.55,0.100,6.800,0.0,19.909091,18.0,18.0,7.0,4.0,4.0,0.0,0.0,10.0,0.00,0.00,1174.0,1110.0,295.0,19.909091,6.700,16.50

In [38]:
Gewicht_Scoreergebnis.shape

(63525, 101)

In [39]:
Gewicht_Scoreergebnis = Gewicht_Scoreergebnis.astype({"gewicht_real": float, "gewicht_pan": float,
                                                    
                                                      "pred_PGU_20": int, "p_PGU_20": float,
                                                      
                                                      "pred_weight_20" : float, 
                                                      
                                                   
                                                      
                                                      
                                                      'sendungs_code' : str,  'ereignis_datum'  : str })

drop_col = [ 'gewicht_quelle','product_code','volumen',	'laenge',	'breite',	'hoehe',	'anzahl_messungen',	'pza_datum',	'ziel_pze_agnr',	'status_pmes_1',	'status_pmes_2',
            'parcel_shape','volumen_grp',	'anzahl_gm',	'd_anzahl_gm',	'd_hoehe_gm',	'd_laenge_gm',	'd_breite_gm',	'd_gewicht_real_gm',	'd_gewicht_pan_gm',	
           'd_volumen_gm',	'd_volumengewicht_real_gm',	'd_volumengewicht_pan_gm',	'd_lbh_summe_gm',	'd_volumengewicht_real2_gm',	'd_volumengewicht_pan2_gm',	'd_gewichtsabweichung_gm',
           'anzahl',	'd_anzahl',	'd_hoehe',	'd_laenge',	'd_breite',	'd_gewicht',	'd_volumen', 'd_lbh_summe',	'd_volumengewicht_pan',	'd_volumengewicht_pan2',	'd_volumengewicht_pan3',
           'vh_laenge_hoehe','vh_laenge_breite', 'vh_laenge_dlaenge', 'vh_breite_dbreite','vh_hoehe_breite','vh_hoehe_dhoehe','vh_hoehe_dhoehe	', 'vh_volumen_dvolumen' ,'vh_gewicht_dgewicht', 
           'vh_gewicht_dgewicht_pan', 'dgewichtsabweichung3', 'dgewichtsabweichung4', 'dgewichtsabweichung5', 
           'dgewichtsabweichung6', 'lbh_summe', 'lbh_vh_gm', 
           'volumengewicht_pan', 'volumengewicht_pan3','anzahl_land_flag_ratio']

Gewicht_Scoreergebnis.drop(drop_col, inplace=True, axis=1, errors='ignore')
print(len(Gewicht_Scoreergebnis))
Gewicht_Scoreergebnis.head()


63525


,sendungs_code,pz_nr,gewicht_real,ereignis_datum,gewicht_pan,country_code,flag_gewicht_20,flag_gewicht_10,d_hoehe_min_gm,d_hoehe_max_gm,d_laenge_min_gm,d_laenge_max_gm,d_breite_min_gm,d_breite_max_gm,d_gewicht_real_min_gm,d_gewicht_real_max_gm,d_gewicht_pan_min_gm,d_gewicht_pan_max_gm,d_volumen_min_gm,d_volumen_max_gm,anzahl_gu_20,anzahl_gu_10,d_anteil_gu_20,d_anteil_gu_10,d_laenge_diff_gm,d_breite_diff_gm,d_hoehe_diff_gm,d_volumen_diff_gm,d_gewicht_pan_diff_gm,d_gewicht_real_diff_gm,d_volumengewicht_diff_gm,d_hoehe_min,d_hoehe_max,d_laenge_min,d_laenge_max,d_breite_min,d_breite_max,d_gewicht_min,d_gewicht_max,d_volumen_min,d_volumen_max,volumengewicht_pan2,pred_PGU_20,p_PGU_20,pred_weight_20
0,CA485224047AT,60.0,0.50,None,NaN,AT,0,0,0.0,490.0,0.0,1284.0,0.0,808.0,0.02,32.10,0.001,9.999,0.0,19.666667,18.0,496.0,0.00,0.00,1284.0,808.0,490.0,19.666667,9.998,32.08,0.0,0.0,612.0,0.0,6128.0,0.0,863.0,0.001,230.00,0.0,19.800000,11.0,0,0.0,1.125576
1,CC068797645FR,43.0,0.35,None,NaN,FR,0,0,0.0,412.0,0.0,1267.0,0.0,840.0,0.02,27.50,0.001,9.430,0.0,19.666667,7.0,447.0,0.00,0.00,1267.0,840.0,412.0,19.666667,9.429,27.48,0.0,0.0,534.0,0.0,1820.0,0.0,887.0,0.001,31.50,0.0,19.666667,33.0,0,0.0,0.769074
2,CM002214159NL,90.0,2.45,2024-08-24,2.44,NL,0,0,0.0,320.0,0.0,1254.0,0.0,945.0,0.05,24.00,0.005,9.896,0.0,19.750000,6.0,37.0,0.00,0.00,1254.0,945.0,320.0,19.750000,9.891,23.95,0.0,0.0,588.0,0.0,6656.0,0.0,880.0,0.001,30.34,0.0,19.500000,15.0,0,0.0,2.178725
3,CP297144822PL,43.0,1.30,None,NaN,PL,0,0,0.0,325.0,0.0,1136.0,0.0,771.0,0.05,24.10,0.010,9.980,0.0,19.600000,2.0,43.0,0.00,0.00,1136.0,771.0,325.0,19.600000,9.970,24.05,0.0,0.0,347.0,0.0,1223.0,0.0,853.0,0.010,23.00,0.0,19.666667,25.0,0,0.0,1.875646
4,CC642097749SE,25.0,3.20,None,NaN,SE,0,0,0.0,295.0,0.0,1174.0,0.0,1110.0,0.05,16.55,0.100,6.800,0.0,19.909091,0.0,10.0,0.00,0.00,1174.0,1110.0,295.0,19.909091,6.700,16.50,0.0,0.0,284.0,0.0,1072.0,0.0,511.0,0.100,16.55,0.0,19.000000,14.0,0,0.0,2.865466


In [40]:
Gewicht_Scoreergebnis_export = Gewicht_Scoreergebnis[['sendungs_code', 'gewicht_pan',   'gewicht_real' ,        
        'flag_gewicht_20' ,             
        'flag_gewicht_10',       
        'ereignis_datum',      
        'pred_PGU_20' ,          
        'p_PGU_20',
        'pred_weight_20'] ]           

In [41]:
Zieltabelle = "Herkules_int_Scorerg_valid_1"
Index = 'sendungs_code'
 
# Falls notwendig löschen der neuen Tabelle
drop_table(f'''DBX_DWH_SBX_GB23_PRD.{Zieltabelle}''')

# Erstellen der neuen Tabelle    
td_connection.execute_sql(f'''
    CREATE MULTISET TABLE DBX_DWH_SBX_GB23_PRD.{Zieltabelle}(
        sendungs_code         varchar(42),
        gewicht_pan           float,
        gewicht_real          float,
        flag_gewicht_20       integer,        
        flag_gewicht_10       integer,
        ereignis_datum        varchar(25),
        pred_PGU_20           integer,
        p_PGU_20              float,
        pred_weight_20        float
    )PRIMARY INDEX({Index})''')
 
print(f'''DBX_DWH_SBX_GB23_PRD.{Zieltabelle} initialised with PI {Index}.''')


# Upload
 
td_connection._upload_df(schema='DBX_DWH_SBX_GB23_PRD', table_name=Zieltabelle, df=Gewicht_Scoreergebnis_export)
print(f'''Data uploaded into table {Zieltabelle}.''')
display(td_connection.select(f'''select count(1) from DBX_DWH_SBX_GB23_PRD.{Zieltabelle}'''))

DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_valid_1 dropped.
DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_valid_1 initialised with PI sendungs_code.


/opt/shared_environments/reporting/lib/python3.11/site-packages/pnp_connectors/connectors/teradata.py:369: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df[object_columns] = _df[object_columns].replace(np.nan, None)


Data uploaded into table Herkules_int_Scorerg_valid_1.


,Count(1)
0,63525


In [44]:
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_valid_2'
SQL = f'''
   SELECT 	a.*, 
            case   
                    
                    when (p_PGU_20 >= 0.50           )     then 1 
                    when (p_PGU_20 >= 0.45         and   pred_weight_20 > 20.5   )     then 1
                    when (p_PGU_20 >= 0.40         and   pred_weight_20 > 21   )     then 1
                    when (p_PGU_20 >= 0.35         and   pred_weight_20 > 21.5   )     then 1
                    when (p_PGU_20 >= 0.30         and   pred_weight_20 > 22   )     then 1
                    when (p_PGU_20 >= 0.25         and   pred_weight_20 > 22.5   )     then 1
                    when (p_PGU_20 >= 0.20         and   pred_weight_20 > 23   )     then 1   
                    when (p_PGU_20 >= 0.10         and   pred_weight_20 > 24  )     then 1 
                    when pred_weight_20 >  25 then 1
                    else 0
            end as Flag_weight_model_20,
   
            case
                when gewicht_real >= 20 then 1
                else 0
            end as FLAG_weight_greater_real_20
            
   FROM DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_valid_1 as a

    '''
Index = 'Sendungs_code'
create_table(Tabelle, SQL, Index)


#RF results
display(td_connection.select(f'''SELECT 
    (SUM(CASE WHEN FLAG_weight_greater_real_20 = pred_PGU_20 THEN 1 ELSE 0 END) * 1.0000 / COUNT(sendungs_code)) AS accuracy from {Tabelle} '''))
display(td_connection.select(f'''select sum(pred_PGU_20*FLAG_weight_greater_real_20*1.0000) / sum(pred_PGU_20*1.00) as Relevanz_PGU_model from {Tabelle} '''))
display(td_connection.select(f'''select sum(pred_PGU_20*FLAG_weight_greater_real_20*1.0000) / sum(FLAG_weight_greater_real_20*1.00) as Trefferquote_PGU_model from {Tabelle} '''))
display(td_connection.select(f'''select pred_PGU_20, FLAG_weight_greater_real_20, count(sendungs_code) as Anzahl from {Tabelle} group by 1, 2 order by 1, 2'''))

#combined model results
display(td_connection.select(f'''select  sum(Flag_weight_model_20*FLAG_weight_greater_real_20*1.0000) / sum(Flag_weight_model_20*1.0000) as Relevanz_combined_models from {Tabelle} '''))
display(td_connection.select(f'''select  sum(Flag_weight_model_20*FLAG_weight_greater_real_20*1.0000) / sum(FLAG_weight_greater_real_20*1.0000) as Trefferquote_combined_model from {Tabelle} '''))
display(td_connection.select(f'''select Flag_weight_model_20, FLAG_weight_greater_real_20, count(sendungs_code) as Anzahl from {Tabelle} group by 1, 2 order by 1, 2'''))


DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_valid_2 dropped.
DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_valid_2 initialised with PI Sendungs_code
63525 DS injected into table DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_valid_2.
 


,accuracy
0,0.9925


,Relevanz_PGU_model
0,0.8290


,Trefferquote_PGU_model
0,0.4051


,pred_PGU_20,FLAG_weight_greater_real_20,Anzahl
0,0,0,62760
1,0,1,420
2,1,0,59
3,1,1,286


,Relevanz_combined_models
0,0.8266


,Trefferquote_combined_model
0,0.4051


,Flag_weight_model_20,FLAG_weight_greater_real_20,Anzahl
0,0,0,62759
1,0,1,420
2,1,0,60
3,1,1,286


In [43]:
now = date.today()
day_akt = now.day 
jahr_akt = now.year 
monat_akt = now.month 

Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_Report_Modelguete_tmp'

SQL = f'''

   SELECT  'Herkules_Import_parcels_GK_20' as Algorithm_name,
   
   
           {jahr_akt} as Jahr,

           {monat_akt} as Monat, 
           
           {day_akt} as Day_of_month,
        

           sum(Flag_weight_model_20*FLAG_weight_greater_real_20*1.00) / sum(Flag_weight_model_20*1.00) as Relevanz_combined_models,

           sum(Flag_weight_model_20*FLAG_weight_greater_real_20*1.00) / sum(FLAG_weight_greater_real_20*1.00) as Trefferquote_combined_model,

           

           sum(pred_PGU_20*FLAG_weight_greater_real_20*1.00) / sum(pred_PGU_20*1.00) as Relevanz_PGU_model,

           sum(pred_PGU_20*FLAG_weight_greater_real_20*1.00) / sum(FLAG_weight_greater_real_20*1.00) as Trefferquote_PGU_model

           

   FROM DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_valid_2 AS a

 
    '''

Index = 'Jahr, Monat,Day_of_month'

create_table(Tabelle, SQL, Index)

display(td_connection.select(f'''select * from {Tabelle} '''))

DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_Report_Modelguete_tmp dropped.
DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_Report_Modelguete_tmp initialised with PI Jahr, Monat,Day_of_month
1 DS injected into table DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_Report_Modelguete_tmp.
 


,Algorithm_name,Jahr,Monat,Day_of_month,Relevanz_combined_models,Trefferquote_combined_model,Relevanz_PGU_model,Trefferquote_PGU_model
0,Herkules_Import_parcels_GK_20,2024,9,8,0.83,0.41,0.83,0.41


In [173]:
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_Report_Modelguete_tmp2'

SQL = f'''

   SELECT  *

   FROM DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_Report_Modelguete_tmp 
   
   union all 
   
   SELECT  *
   
   FROM DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_Report_Modelguete 

 
    '''

Index = 'Jahr, Monat,Day_of_month'

create_table(Tabelle, SQL, Index)

display(td_connection.select(f'''select * from {Tabelle} order by 3,4'''))

DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_Report_Modelguete_tmp2 dropped.
DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_Report_Modelguete_tmp2 initialised with PI Jahr, Monat,Day_of_month
5 DS injected into table DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_Report_Modelguete_tmp2.
 


,Algorithm_name,Jahr,Monat,Day_of_month,Relevanz_combined_models,Trefferquote_combined_model,Relevanz_PGU_model,Trefferquote_PGU_model
0,Herkules_Import_parcels_GK_20,2024,7,9,0.94,0.20,0.85,0.27
1,Herkules_Import_parcels_GK_20,2024,7,9,0.94,0.20,0.85,0.27
2,Herkules_Import_parcels_GK_20,2024,7,31,0.71,0.29,0.74,0.29
3,Herkules_Import_parcels_GK_20,2024,7,31,0.92,0.18,0.74,0.29
4,Herkules_Import_parcels_GK_20,2024,7,31,0.71,0.29,0.74,0.29


In [174]:
Tabelle = 'DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_Report_Modelguete'

SQL = f'''

   SELECT  *
           
   FROM DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_Report_Modelguete_tmp2
   
   
 
    '''

Index = 'Jahr, Monat,Day_of_month'

create_table(Tabelle, SQL, Index)

display(td_connection.select(f'''select * from {Tabelle} '''))

DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_Report_Modelguete dropped.
DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_Report_Modelguete initialised with PI Jahr, Monat,Day_of_month
5 DS injected into table DBX_DWH_SBX_GB23_PRD.Herkules_int_Scorerg_Report_Modelguete.
 


,Algorithm_name,Jahr,Monat,Day_of_month,Relevanz_combined_models,Trefferquote_combined_model,Relevanz_PGU_model,Trefferquote_PGU_model
0,Herkules_Import_parcels_GK_20,2024,7,31,0.71,0.29,0.74,0.29
1,Herkules_Import_parcels_GK_20,2024,7,9,0.94,0.20,0.85,0.27
2,Herkules_Import_parcels_GK_20,2024,7,31,0.92,0.18,0.74,0.29
3,Herkules_Import_parcels_GK_20,2024,7,9,0.94,0.20,0.85,0.27
4,Herkules_Import_parcels_GK_20,2024,7,31,0.71,0.29,0.74,0.29


# ENDE !!!!

In [ ]:
## Findings

# analyse this parcel pan data 'JD000030171762000001698512' pan date and pz date

# analyse this parce PZA weight and PZE weight 'JD000030072307000032767800'

# for this parcel 'JATA8214310002919826' we have pan weight but not real weight

# this parcel 'JD149990200018055307' is national 

In [ ]:
# Take LFT parcels
#gewicht_quelle = 'W' or'Y'